# More ways to access with variables

The stream-lined way to input variables to `ocean_data_gateway` is demonstrated in the "Details" section. However, it is still possible to input variables individually by reader. That is demonstrated here.

In [1]:
from cf_xarray.units import units  # isort:skip
import pint_xarray  # isort:skip

pint_xarray.unit_registry = units  # isort:skip
import ocean_data_gateway as odg
import pandas as pd
import xarray as xr
import numpy as np
pd.set_option('display.max_rows', 5)

In [2]:
kw = {
    "min_lon": -124.0,
    "max_lon": -123.0,
    "min_lat": 39.0,
    "max_lat": 40.0,
    "min_time": '2021-4-1',
    "max_time": '2021-4-2',
}

### All variables

Don't input anything with the `variables` keyword, or use `'variables': None`:

In [3]:
kwargs = {
          'kw': kw, 
          'approach': 'region',
          'readers': [odg.erddap,
                      odg.axds],
          'variables': None
}
data = odg.Gateway(**kwargs)

### By variables(s)

If no `variables` are specified for a given reader (for a `region` search), datasets with any variables will be returned from a search.

However, if you want to specify a variable or variables, keep in mind that different readers have different names for variables, which is why you can't just input a variable name for all the readers. 

This is only relevant for the ERDDAP and Axiom readers currently (it will retain all variables in local files). The Axiom reader of type `platform2` will search by variable where the available variable names are specified, and of type `layer_group`, the `query` method will be used for variable searching.

Let's say you want to search for salinity. You can input the base of the word as `variables` ("sal" or "salinity" but not "salt" since the checker searches for matches with the whole input variable name and "salt" isn't used for any variable names) and the code will make sure it exactly matches a known variable name. If it cannot match, it will throw an error with suggestions. This is not done automatically since for example "soil_salinity" matches for "salinity". You need to do this for each `known_server` for the `erddap` reader separately, and specific variables will only be used to filter for the `axds` reader for `axds_type='platform2'`. Any variable names can be input for the `axds` reader for `axds_type='layer_group'`.

You can first check each reader/server individually to determine the variable names you want to use:

In [4]:
# IOOS ERDDAP
server = 'http://erddap.sensors.ioos.us/erddap'
odg.search_variables(server, 'sal')

,count
variable,
salinity,954
salinity_qc,954
...,...
sea_water_practical_salinity_4161sc_a_qc_agg,1
sea_water_practical_salinity_10091sc_a,1


In [5]:
# Coastwatch ERDDAP
server = 'http://coastwatch.pfeg.noaa.gov/erddap'
odg.search_variables(server, 'sal')

,count
variable,
salinity,73
salt,4
...,...
stop_salinity,1
ts_salinity,1


In [6]:
# axds 'platform2'
odg.search_variables('axds', 'sal')

,count
variable,
Salinity,3204
Soil Salinity,622


Once you have selected variables that match, the code won't complain anymore.

In [7]:
kwargs = {
          'kw': kw, 
          'approach': 'region',
          'readers': [odg.erddap,
                      odg.axds],

          'erddap': {
                          'known_server': ['coastwatch','ioos'],
                           'variables': [['salinity', 'sea_water_salinity'],
                                         ['salinity', 'sea_water_practical_salinity']]
          },
          'axds': {
                          'axds_type': ['platform2','layer_group'],
                         'variables': ['Salinity',
                                       'salinity'  # this one can be called anything that might make a match
                                      ]},
}

data = odg.Gateway(**kwargs)

In [8]:
data.dataset_ids

['d359748a-fe78-11e7-8128-0023aeec7b98',
 '5104d464-8a30-4720-aeb7-57e801844e6e',
 '3261285c-e3c9-45fd-b777-e6d681a3eaad']

### Other actions with variables

Demonstrated here are the available functions to work with variables. Currently the ways to call the individiual libraries aren't pretty but they'll work. Note that the number of times a variable is used in the system is also included under "count" to see what the popular names are (many are not widely used). 


#### All available variables

Return all variables for the two ERDDAP `known_server`s, then for the Axiom reader `axds_type='platform2'`.

In [9]:
# IOOS ERDDAP
server = 'http://erddap.sensors.ioos.us/erddap'
odg.all_variables(server)

,count
variable,
air_pressure,4028
air_pressure_10011met_a,2
...,...
wmo_id,954
z,37377


In [10]:
# Coastwatch ERDDAP
server = 'http://coastwatch.pfeg.noaa.gov/erddap'
odg.all_variables(server)

,count
variable,
abund_m3,2
ac_line,1
...,...
z_mean,2
zlev,6


The Axiom reader variables are for `axds_type='platform2'` not `axds_type='layer_group` since the latter are more unique grid products that don't conform well.

In [11]:
# axds 'platform2'
odg.all_variables('axds')

,count
variable,
Ammonium,23
Atmospheric Pressure: Air Pressure at Sea Level,362
...,...
Winds: at 10 m,18
pH,965


#### All available variables, sorted by count

In [12]:
# IOOS ERDDAP
server = 'http://erddap.sensors.ioos.us/erddap'
odg.search_variables(server, '')

,count
variable,
time,38331
longitude,38331
...,...
sea_surface_wave_from_direction_elw11b2t01wv_qc_tests,1
sea_water_pressure_7263arc_a,1


In [13]:
# Coastwatch ERDDAP
server = 'http://coastwatch.pfeg.noaa.gov/erddap'
odg.search_variables(server, '')

,count
variable,
time,1637
longitude,1352
...,...
haliotis_fulgens_stderr,1
observationuri,1


In [14]:
# axds 'platform2'
odg.search_variables('axds', '')

,count
variable,
Stream Height,19758
Water Surface above Datum,19489
...,...
CO2: PPM of Carbon Dioxide in Air in Dry Gas,1
Evaporation Rate,1


#### Variables search, sorted by count

In [15]:
# IOOS ERDDAP
server = 'http://erddap.sensors.ioos.us/erddap'
odg.search_variables(server, 'sal')

,count
variable,
salinity,954
salinity_qc,954
...,...
sea_water_practical_salinity_4161sc_a_qc_agg,1
sea_water_practical_salinity_10091sc_a,1


In [16]:
# Coastwatch ERDDAP
server = 'http://coastwatch.pfeg.noaa.gov/erddap'
odg.search_variables(server, 'sal')

,count
variable,
salinity,73
salt,4
...,...
stop_salinity,1
ts_salinity,1


In [17]:
# axds 'platform2'
odg.search_variables('axds', 'sal')

,count
variable,
Salinity,3204
Soil Salinity,622


#### Check variables

And finally you can check to make sure you have good variables. No news is good news in this. Reminder that you don't check for axds reader for axds_type='layer_group' because that is searched for in the database just by name as a query.

In [18]:
# IOOS ERDDAP
server = 'http://erddap.sensors.ioos.us/erddap'
odg.check_variables(server, ['salinity', 'sea_water_practical_salinity'])

In [19]:
# Coastwatch ERDDAP
server = 'http://coastwatch.pfeg.noaa.gov/erddap'
odg.check_variables(server, ['salinity', 'sea_water_salinity'])

In [20]:
# axds 'platform2'
odg.check_variables('axds', 'Salinity')